# Mutation-Based Fuzzing

Most [randomly generated inputs](Basic_Fuzzing.ipynb) are syntactically _invalid_ and thus are quickly rejected by the processing program.  To exercise functionality beyond input processing, we must increase chances to obtain valid inputs.  One such way is by _mutating_ existing valid inputs - that is, introducing small changes that may still keep the input valid, yet exercise new behavior.

**Prerequisites**

* You should know how basic fuzzing works; for instance, from the ["Fuzzing"](Basic_Fuzzing.ipynb) chapter.

## The Problem

Most modern programs do a good job in _validating_ their inputs before they actually process them.  As an example, think of a _compiler_ translating program code into a lower-level language.  The processing steps of a compiler are typically depicted as a pipeline of components, each processing the input of its predecessors and producing output for its successors.  In the beginning, we typically have a lexical analysis that puts letters together into words, a syntactic analysis that puts sequences of words and items into structures, and then the actual compilation steps that translate these structures into code:

![Compiler Pipeline](PICS/Compiler.pdf)

The problem is that with random inputs, you will be able to exercise a lot of functionality in the leftmost stages (i.e., the lexical and possibly syntactical analyses), but the chances of actually producing a _valid_ input that will make it to the later stages are slim.


### Fuzzing Python Expressions

To illustrate just how low the chances are, let us _illustrate the problem on the Python interpreter._  Let us get the fuzzing function from  the ["Fuzzing"](Basic_Fuzzing.ipynb) chapter.

In [687]:
import gstbook
from Basic_Fuzzing import fuzzer

We can use the `fuzzer()` function to generate random inputs.  Would this be a valid Python expression?

In [688]:
fuzzer()

'546&%-$>*:$7$::(29\'7$<32=(&(1-.\':.:\'=;-;#:#1$02&"-$21+?;&-29'

To test which inputs are actually valid, we use the Python `parser` module.  `parser.suite(_source_)` returns an internal object (actually, a parse tree) if _source_ is a valid command:

In [689]:
import parser

parser.suite("print(2 + 2)")

Note that the command is _not_ executed.  That is because evaluating or otherwise executing randomly generated strings would be quite a risk: What happens if by chance, we create a command that deletes your files?

If _source_ is invalid, `parser.suite()` raises an exception:

In [690]:
from ExpectError import ExpectError

with ExpectError():
    parser.suite("print(2 +<>^& 37)")

Traceback (most recent call last):
  File "<ipython-input-690-69f39c18ad19>", line 4, in <module>
    parser.suite("print(2 +<>^& 37)")
  File "<string>", line 1
    print(2 +<>^& 37)
              ^
SyntaxError: invalid syntax


We can thus write a function `is_valid_expr()` that checks whether an expression is valid in Python:

In [691]:
def is_valid_expr(source):
    """Returns true iff source is a valid Python expression"""
    if '#' in source:
        return False
    try:
        parser.suite("print(" + source + ")")
        return True
    except SyntaxError:
        return False
    except ValueError:
        return False

In [692]:
assert is_valid_expr("4 + 4")
assert not is_valid_expr("37 !@#$ 564")

### Invalid Inputs

Let us see how many of the `fuzzer()` outputs are actually valid:

In [693]:
valid_inputs = set()
TRIALS = 1000

for i in range(TRIALS):
    input = fuzzer()
    if is_valid_expr(input):
        valid_inputs.add(input)

len(valid_inputs) / TRIALS

0.009

About 99% of all generated inputs are invalid - that's not very many.  What are the valid ones we get?

In [694]:
valid_inputs

{'', ' ', '"(/-3=25(0"', '0', '1', '24', '6.', '8', '9,161'}

While we do have a chance to create numbers and very simple arithmetic expressions, we are going to miss plenty of Python data types and functionality - and of course, we are not going to cover the code that handles these.  

What are the odds of producing a Python set, for instance? It would have to start with `set(`.  In its default configuration, for instance, `get_fuzzer()` does not even produce letters.  If we give it a range of say, 64 characters, we have a chance of $1 : 64^4$ to have an input that starts with `"set("`.  How much is that, again?

In [695]:
64 ** 4

16777216

Indeed, less than one in a million.  Plus, we'd also need the closing `')'` character...

## Mutating Inputs

The alternative to generating random strings from scratch is to start with a guiven _valid_ input, and then to subsequently _mutate_ it.  A _mutation_ in this context is a simple string manipulation - say, inserting a (random) character, deleting a character, or flipping a bit in a character representation.  Here are some mutations to get you started:

In [696]:
import random

In [697]:
def delete_random_character(s):
    """Returns s with a random character deleted"""
    if s == "":
        return s

    pos = random.randint(0, len(s) - 1)
    # print("Deleting", repr(s[pos]), "at", pos)
    return s[:pos] + s[pos + 1:]

In [698]:
for i in range(10):
    x = delete_random_character("A quick brown fox")
    print(x)

A quck brown fox
A quik brown fox
A quck brown fox
A quick brown fx
A quck brown fox
A quic brown fox
A quck brown fox
A uick brown fox
A qick brown fox
A quick rown fox


In [699]:
def insert_random_character(s):
    """Returns s with a random character inserted"""
    pos = random.randint(0, len(s))
    random_character = chr(random.randrange(32, 64))
    # print("Inserting", repr(random_character), "at", pos)
    return s[:pos] + random_character + s[pos:]

In [700]:
for i in range(10):
    print(insert_random_character("A quick brown fox"))

A$ quick brown fox
A quick brown fo6x
A- quick brown fox
A qu6ick brown fox
A qu7ick brown fox
A quick brown fox%
A quick br4own fox
A quick brown fo$x
A0 quick brown fox
A quick brow$n fox


In [701]:
def flip_random_character(s):
    """Returns s with a random bit flipped in a random position"""
    if s == "":
        return s

    pos = random.randint(0, len(s) - 1)
    c = s[pos]
    bit = 1 << random.randint(0, 6)
    new_c = chr(ord(c) ^ bit)
    # print("Flipping", bit, "in", repr(c) + ", giving", repr(new_c))
    return s[:pos] + new_c + s[pos + 1:]


In [702]:
for i in range(10):
    print(flip_random_character("A quick brown fox"))

A quyck brown fox
A quick browl fox
A qui#k brown fox
A quick frown fox
A quick brown!fox
A quick brkwn fox
A quick bRown fox
A quick bro7n fox
 quick brown fox
A$quick brown fox


Let us now create a random mutator that randomly chooses which mutation to apply:

In [703]:
mutators = [delete_random_character, insert_random_character, flip_random_character]

In [708]:
def mutate(s):
    """Return s with a random mutation applied"""
    mutator = random.choice(mutators)
    # print(mutator)
    return mutator(s)

In [709]:
for i in range(10):
    print(mutate("A quick brown fox"))

A qtick brown fox
A quicK brown fox
A uuick brown fox
A`quick brown fox
A quigk brown fox
A qick brown fox
A quck brown fox
<A quick brown fox
A quick brown fo)x
A quick bro:wn fox


Let us now apply the `mutate()` function on a Python expression and see how many valid inputs we obtain.

In [705]:
seed_input = "1 + 2 * 3 / 4"
valid_inputs = set()
TRIALS = 1000

for i in range(TRIALS):
    input = mutate(seed_input)
    if is_valid_expr(input):
        valid_inputs.add(input)

The first thing we observe is that the number of valid inputs is now much higher.  We are still far away from 100% valid inputs, but this is an improvement:

In [706]:
len(valid_inputs) / TRIALS

0.096

Most important, though, is that the valid inputs now cover many more Python expression features – additional operands, identifiers, and more:

In [707]:
print(valid_inputs)

{'1 + 2 * 3 / 0', '1 + 2 * 3 / 4.', '9 + 2 * 3 / 4', '1 + 22 * 3 / 4', '1 + r * 3 / 4', '0 + 2 * 3 / 4', '1 + 2 * 3 / 6', '1 + 2 * 83 / 4', '1< + 2 * 3 / 4', '1 + .2 * 3 / 4', '1 + 3 * 3 / 4', '1 + 2 * 31 / 4', '1 + 2 * 3 / 24', '1 + 2 * 13 / 4', '12 + 2 * 3 / 4', ' 1 + 2 * 3 / 4', '1 + 2 * 30 / 4', '1 + 2 * 3/ 4', '1 + 2 * 38 / 4', '1 + 2 * 3 /  4', '51 + 2 * 3 / 4', '1 + 21 * 3 / 4', '1 / 2 * 3 / 4', '1 + 2 * 2 / 4', '1 >+ 2 * 3 / 4', '1 =+ 2 * 3 / 4', '10+ 2 * 3 / 4', '1 + 2 * 7 / 4', '1 + 2 * 36 / 4', '1 *+ 2 * 3 / 4', '1 + 0 * 3 / 4', '1 /+ 2 * 3 / 4', '1 + 2 * s / 4', '1 + 2 * 3 /4', '1 + 24 * 3 / 4', '1 + 2 *3 / 4', '1 + 2 * 34 / 4', '1 + 2 * 1 / 4', '1 + 2 * 3  / 4', '1 + 27 * 3 / 4', '1 + 2 + 3 / 4', '1 * 2 * 3 / 4', '1 + 2 * 3 / -4', '3 + 2 * 3 / 4', '1 + 2 * 3 + 4', '1 ,+ 2 * 3 / 4', '1 + 2 * 93 / 4', ' + 2 * 3 / 4', '1. + 2 * 3 / 4', '1 + 2  * 3 / 4', '1 + 2 * 3 - 4', '1 + 2 * 3 / 4,', 'q + 2 * 3 / 4', '16 + 2 * 3 / 4', '1 + 2 * 30/ 4', '1  + 2 * 3 / 4', '1 + 2 *+ 3 / 4', '

## Mutation Fuzzing vs Basic Fuzzing

\todo{Compare coverage}

## Next Steps


\todo{Rework}

How can we sufficiently cover functionality?  Essentially, we have two options:

1. Try to cover as much _implemented_ functionality as possible.  To this end, we need to access the program implementation, measure which parts would actually be reached with our inputs, and use this _coverage_ to guide our search.  We will explore this in the next chapter, which discusses [coverage](Coverage.ipynb).

2. Try to cover as much _specified_ functionality as possible.  Here, we would need a specification of the input format, distinguishing between individual input elements such as (in our case) numbers, operators, comments, and strings – and attempting to cover as many of these as possible.  We will explore this as it comes to [grammar-based testing](Grammar_Testing.ipynb).

Finally, the concept of a "population" that is systematically "evolved" trhough "mutations" will be explored in depth when discussing [search-based testing](Search_Based_Testing.ipynb).  Enjoy!


## Exercises


### Exercise 1

Apply the above mutation-based fuzzing technique on `bc`, using files, as in our [Introduction to Fuzzing](Basic_Fuzzing.ipynb).